In [3]:
import os
import glob
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt
from imgaug import augmenters as iaa
import imgaug as ia
from scipy.spatial.distance import cosine, hamming, jaccard, cdist
from sklearn.metrics import f1_score as calc_f1
from sklearn.metrics import precision_score, recall_score
# f1_score = calc_f1(y_true, y_pred, labels=None, pos_label=1, average=’binary’, sample_weight=None)
pd.set_option('display.max_rows', 2000)

In [8]:
hash_size = 15
high_freq_factor = 4

n_film = 3
wd_frames = os.getcwd() + '/films/film_{}/frames/'.format(n_film)
frame_paths = sorted(glob.glob(wd_frames + '*'))

wd_film = os.getcwd() + '/films/film_{}/film/'.format(n_film)
film_name = glob.glob(path_film + '*')[0].split('/')[-1].replace(' ', '-')[0:7]

wd_hashes = os.getcwd() + '/films/film_{}/hashes/'.format(n_film)

In [20]:
with open(wd_hashes + '{}-hashes-{}-{}.csv'.format(film_name, hash_size, high_freq_factor)) as csvfile:
    dataset = pd.read_csv(csvfile, index_col = False)

In [25]:
dataset['annotation'] = np.random.choice(2, len(dataset.hash))

array([1, 0, 0, 1, 1, 1, 1, 0, 0, 0])

In [18]:
dataset = dataset.drop(columns = 'Unnamed: 0')
dataset = dataset.drop(columns = 'Unnamed: 0.1')
dataset = dataset.drop(columns = 'Unnamed: 0.1.1')

In [5]:
def zero_padding_numbers(frame_name, len_number=4, frame_nr = False):
    if frame_nr == False:
        frame_nr = frame_name.split('frame')[1].split('.')[0]
        start = frame_name.split(frame_nr)[0]
        end = frame_name.split(frame_nr)[1]
    else:
        start = ''
        frame_nr = str(frame_name)
        end = ''
    len_padding = len_number-len(frame_nr)
    new_nr = ('0'*len_padding) + frame_nr
    new_name = start + new_nr + end
    return new_name

In [6]:
#shot annotation
shot1 = list(range(840, 1508))
shot2 = list(range(1528, 1911))
shot3 = list(range(2611, 2674))
shot4 = list(range(3280, 3461))
shot5 = list(range(4453, 4612))

shots = shot1 + shot2 + shot3 + shot4 + shot5
shot_numbers = [zero_padding_numbers(shot, frame_nr = True) for shot in shots]
correct_frames = ['frame{}.jpg'.format(shot_number) for shot_number in shot_numbers]

annotation = []
for frame in list(dataset.frame):
    if frame in correct_frames:
        annotation.append(1)
    else:
        annotation.append(0)
dataset['annotation'] = annotation

In [8]:
dataset = dataset.sort_values(by = 'frame')
# change hashtype to numpy array
dataset['hash'] = dataset['hash'].apply(lambda x: np.array([int(e) for e in x if e =='0' or e =='1']))

In [ ]:
# add zero padding to frame name numbering
dataset['frame'] = dataset.frame.apply(lambda x: zero_padding_numbers(x))
dataset = dataset.sort_values(by = 'frame')
# change hashtype to numpy array
dataset['hash'] = dataset['hash'].apply(lambda x: np.array([int(e) for e in x if e =='0' or e =='1']))

In [10]:
os.chdir(wd + '/films/film_1/hashes/')


In [20]:
dataset.to_csv('SeeFoo-hashes-15-4.csv')

In [ ]:
def get_trailer_frames(frames, len_number = 4):
    def zero_padding_numbers(frame_name, len_number=4, frame_nr = False):
        if frame_nr == False:
            frame_nr = frame_name.split('frame')[1].split('.')[0]
            start = frame_name.split(frame_nr)[0]
            end = frame_name.split(frame_nr)[1]
        else:
            start = ''
            frame_nr = str(frame_name)
            end = ''
        len_padding = len_number-len(frame_nr)
        new_nr = ('0'*len_padding) + frame_nr
        new_name = start + new_nr + end
        return new_name

    shot1 = list(range(840, 1508))
    shot2 = list(range(1528, 1911))
    shot3 = list(range(2611, 2674))
    shot4 = list(range(3280, 3461))
    shot5 = list(range(4453, 4612))

    shots = shot1 + shot2 + shot3 + shot4 + shot5
    shot_numbers = [zero_padding_numbers(shot, frame_nr = True) for shot in shots]
    correct_frames = ['frame{}.jpg'.format(shot_number) for shot_number in shot_numbers]
    trailer_frames = [frame for frame in frames if frame.split('/')[-1] in correct_frames]    
    return trailer_frames